# Importante 

* Tener en cuenta bajadas de Procedimientos, Personas Armas, Divisas , vehiculos (secuestrados, ministerio), Narcotrafico general,Objetos y pegar en la carpeta bajadas de fecha del primero del 1 del mes en informar hasta fecha del informe 

### IMPORTACIÓN DE LIBREARIAS


In [1]:
import pandas as pd
from  Parametros import * 
from  Funciones import * 
from openpyxl import load_workbook


### ACCEDER BASE DE DATOS

In [2]:
df_base_procedimientos = pd.read_excel("db/Base_procedimientos_2023_2024.xlsx")
df_base_operaciones = pd.read_excel("db/Base_operaciones_2023_2024.xlsx")


### PROCEDIMIENTOS GENERALES




In [3]:
excel_bajada_procedimientos_generales = filtrar_procedimientos_generales("bajadas/bajada_general.xls")

df_geog = pd.DataFrame()
df_geog['UID'] = excel_bajada_procedimientos_generales.apply(generar_uid_sigpol, axis=1)
df_geog['ID_OPERATIVO'] = excel_bajada_procedimientos_generales.apply(procesar_causa_judicial, axis=1)
df_geog['FUERZA_INTERVINIENTE'] = "PSA"
df_geog['ID_PROCEDIMIENTO'] = df_geog['UID']
df_geog['CAUSAJUDICIALNUMERO'] = excel_bajada_procedimientos_generales['CAUSAJUDICIALNUMERO'].copy()
df_geog['UNIDAD_INTERVINIENTE'] = excel_bajada_procedimientos_generales['UOSP']
df_geog['DESCRIPCIÓN'] = excel_bajada_procedimientos_generales.apply(procesar_descripcion, axis=1)
df_geog['TIPO_INTERVENCION'] = excel_bajada_procedimientos_generales.apply(procesar_tipo_procedimiento, axis=1)
df_geog['PROVINCIA'] = excel_bajada_procedimientos_generales.apply(procesar_provincia, axis=1)
df_geog['DEPARTAMENTO O PARTIDO'] = excel_bajada_procedimientos_generales.apply(procesar_municipio, axis=1)
df_geog['LOCALIDAD'] = "-"
df_geog['DIRECCION'] = excel_bajada_procedimientos_generales.apply(procesar_direccion, axis=1)
df_geog['LATITUD'] = excel_bajada_procedimientos_generales['GEOREFERENCIA_Y']
df_geog['LONGITUD'] = excel_bajada_procedimientos_generales['GEOREFERENCIA_X']
df_geog['FECHA'] = pd.to_datetime(excel_bajada_procedimientos_generales['DENUNCIAFECHA'], errors='coerce').dt.strftime('%d/%m/%Y')
df_geog['HORA'] = pd.to_datetime(excel_bajada_procedimientos_generales['DENUNCIAFECHA'], errors='coerce').dt.strftime('%H:%M')
df_geog['ZONA_SEGURIDAD_FRONTERAS'] = "-"
df_geog['PASO_FRONTERIZO'] = "-"
df_geog['OTRAS AGENCIAS INTERVINIENTES'] = "-"
df_geog['Observaciones - Detalles'] = "-"

df_geog = df_geog[[ "UID",'FUERZA_INTERVINIENTE', 'ID_OPERATIVO', 'ID_PROCEDIMIENTO',
                                     'UNIDAD_INTERVINIENTE', 'DESCRIPCIÓN', 'TIPO_INTERVENCION',
                                     'PROVINCIA', 'DEPARTAMENTO O PARTIDO', 'LOCALIDAD', 'DIRECCION',
                                     'ZONA_SEGURIDAD_FRONTERAS', 'PASO_FRONTERIZO', 'LATITUD', 'LONGITUD',
                                     'FECHA', 'HORA', 'OTRAS AGENCIAS INTERVINIENTES', 'Observaciones - Detalles']]


df_geog_final = df_geog[~df_geog['UID'].isin(set(df_base_procedimientos['UID']))]
print(f"\nCasos ya informados: {len(df_base_procedimientos)}")
print(f"Casos nuevos para informar : {len(df_geog_final)}")


Estadistica de Partes

Cantidad de Partes inicial: 564
Cantidad Duplicado: 3
Cantidad No diponible: 34
Cantidad de Partes final: 527

Casos ya informados: 8738
Casos nuevos para informar : 92


### HOJA PERSONAS

In [4]:

# Paso 1: Cargar datos iniciales
excel_bajada_personas = pd.read_excel("bajadas/bajada_general_persona.xls")
print(f"Total de registros cargados: {len(excel_bajada_personas)}")

# Paso 2: Generar UID

excel_bajada_personas['TIPO_CAUSA_INTERNA'] = excel_bajada_personas.apply( procesar_tipo_causa_interna, axis=1)
excel_bajada_personas['UID'] = excel_bajada_personas.apply(generar_uid_sigpol, axis=1)

# Paso 3: Coincidencia de UID entre excel_bajada_personas y df_geog_final
uid_coincidentes = excel_bajada_personas[excel_bajada_personas['UID'].isin(df_geog_final['UID'])]
print(f"Cantidad de registros con UID coincidente: {len(uid_coincidentes)}")

# Crear DataFrame de detenidos y aprendidos
df_detenidos_aprendidos = pd.DataFrame()
df_detenidos_aprendidos['UID'] = uid_coincidentes['UID']
df_detenidos_aprendidos['EDAD'] = uid_coincidentes.apply(procesar_edad, axis=1)
df_detenidos_aprendidos['SEXO'] = uid_coincidentes.apply(procesar_sexo, axis=1)
df_detenidos_aprendidos['GENERO'] = uid_coincidentes.apply(procesar_genero, axis=1)
df_detenidos_aprendidos['NACIONALIDAD'] = uid_coincidentes.apply(procesar_nacionalidad, axis=1)
df_detenidos_aprendidos['SITUACION_PROCESAL'] = uid_coincidentes.apply(procesar_situacion_judicial, axis=1)
df_detenidos_aprendidos['DELITO_IMPUTADO'] = uid_coincidentes.apply(procesar_tipo_delito, axis=1)
df_detenidos_aprendidos['JUZGADO_INTERVINIENTE'] = uid_coincidentes.apply(procesar_juzgado, axis=1)
df_detenidos_aprendidos['CARATULA_CAUSA'] = uid_coincidentes.apply(procesar_caratula, axis=1)

print(f"Total de registros después del procesamiento de columnas específicas: {len(df_detenidos_aprendidos)}")

# Paso 4: Filtrar SITUACION_PROCESAL 'NO INFORMAR'
df_detenidos_aprendidos = df_detenidos_aprendidos[df_detenidos_aprendidos['SITUACION_PROCESAL'] != 'NO INFORMAR']
print(f"Cantidad tras exclusión de 'NO INFORMAR': {len(df_detenidos_aprendidos)}")

# Paso 5: Fusionar df_geog_final y df_detenidos_aprendidos en base al UID
df_detenidos_aprendidos = pd.merge(df_geog_final, df_detenidos_aprendidos, on='UID', how='left')
print(f"Total de registros después de la fusión con df_geog_final: {len(df_detenidos_aprendidos)}")

# Paso 6: Relleno de valores vacíos con '-'
df_detenidos_aprendidos = df_detenidos_aprendidos.apply(lambda x: x.fillna('-') if x.dtype == 'object' else x)


# Paso 7: Selección y reordenamiento de columnas
df_detenidos_aprendidos = df_detenidos_aprendidos[[
    "UID",
    'FUERZA_INTERVINIENTE',
    'ID_OPERATIVO',
    'ID_PROCEDIMIENTO',
    'UNIDAD_INTERVINIENTE',
    'DESCRIPCIÓN',
    'TIPO_INTERVENCION',
    'EDAD',
    'SEXO',
    'GENERO',
    'NACIONALIDAD',
    'SITUACION_PROCESAL',
    "DELITO_IMPUTADO",
    'JUZGADO_INTERVINIENTE',
    'CARATULA_CAUSA'
]]

print(f"Total de registros después de seleccionar y reordenar columnas: {len(df_detenidos_aprendidos)}")


Total de registros cargados: 562
Cantidad de registros con UID coincidente: 94
Total de registros después del procesamiento de columnas específicas: 94
Cantidad tras exclusión de 'NO INFORMAR': 85
Total de registros después de la fusión con df_geog_final: 96
Total de registros después de seleccionar y reordenar columnas: 96


### HOJA OTROS DELITOS

In [5]:
# Crear un nuevo dataframe para otro delitos
df_otros_delitos = df_detenidos_aprendidos[df_detenidos_aprendidos['SITUACION_PROCESAL'] == 'VICTIMA']
# Filtro los que son victima de detenidos y aprendidos
df_detenidos_aprendidos_final = df_detenidos_aprendidos[df_detenidos_aprendidos['SITUACION_PROCESAL'] != 'VICTIMA']

# Cambiar el nombre de algunas columnas en el DataFrame de víctimas
df_otros_delitos = df_otros_delitos.rename(columns={
    'DELITO_IMPUTADO': 'TIPO_OTRO_DELITO',
    'GENERO': 'GENERO_VICTIMA',
    'EDAD': 'EDAD_VICTIMA',
}) # type: ignore

df_otros_delitos["OBSERVACIONES"] ="-"
# Seleccionar las columnas deseadas en el DataFrame de víctimas
df_otros_delitos_final = df_otros_delitos[[
    "UID",
    'FUERZA_INTERVINIENTE',
    'ID_OPERATIVO',
    'ID_PROCEDIMIENTO',
    'UNIDAD_INTERVINIENTE',
    'DESCRIPCIÓN',
    'TIPO_INTERVENCION',
    "TIPO_OTRO_DELITO",
    'GENERO_VICTIMA',
    'EDAD_VICTIMA',
    'NACIONALIDAD',
    'OBSERVACIONES'
]]




In [6]:
# Imprimir la cantidad de registros en el DataFrame de otros delitos para verificar
print(f"Cantidad de registros en geog: {len(df_geog_final)}")
print(f"Cantidad de registros en detenidos_aprendidos: {len(df_detenidos_aprendidos_final)}")
print(f"Cantidad de registros en otros_delitos: {len(df_otros_delitos_final)}")


Cantidad de registros en geog: 92
Cantidad de registros en detenidos_aprendidos: 51
Cantidad de registros en otros_delitos: 45


### INCAUTACIONES ARMA

In [ ]:
# Paso 1: Cargar datos iniciales
excel_bajada_arma = pd.read_excel("bajadas/bajada_arma.xls")
print(f"Total de registros cargados: {len(excel_bajada_arma)}")

# Paso 2: Generar UID

excel_bajada_arma['TIPO_CAUSA_INTERNA'] = excel_bajada_arma.apply( procesar_tipo_causa_interna, axis=1)
excel_bajada_arma['UID'] = excel_bajada_arma.apply(generar_uid_sigpol, axis=1)


# Crear DataFrame de arma
df_arma = pd.DataFrame()
df_arma['UID'] = excel_bajada_arma['UID']
df_arma['TIPO_INCAUTACION'] = "ARMAS"
df_arma['TIPO'] = excel_bajada_arma['TIPO_ARMA']
df_arma['SUB_TIPO'] = "-"
df_arma['CANTIDAD'] = excel_bajada_arma.apply(procesar_cantidad_arma, axis=1)
df_arma['MEDIDAS'] = "UNIDADES"
df_arma['AFORO'] = "-"
df_arma['OBSERVACIONES'] = excel_bajada_arma.apply(procesar_observaciones_arma, axis=1)

# Paso 3: Coincidencia de UID entre excel_bajada_arma y df_geog_final
df_arma_final = excel_bajada_arma[excel_bajada_arma['UID'].isin(df_geog_final['UID'])]
print(f"Cantidad de registros con UID coincidente: {len(uid_coincidentes)}")




Total de registros cargados: 38
Cantidad de registros con UID coincidente: 10
nan
nan
CALIBRE: .22
HAFDASA
HAFDASA
MARCA: HAFDASA - CALIBRE: .22
BERSA
BERSA
MARCA: BERSA - CALIBRE: .22
nan
nan
CALIBRE: .38


MARCA:  - CALIBRE: .38


MARCA:  - CALIBRE: .22
GLOCK
GLOCK
MARCA: GLOCK - CALIBRE: 9MM
nan
nan
CALIBRE: 9MM
SMITH & WESSON
SMITH & WESSON
MARCA: SMITH & WESSON - CALIBRE: .32
SIN DETERMINAR
SIN DETERMINAR
MARCA: SIN DETERMINAR - CALIBRE: OTRO CALIBRE
                 UID TIPO_INCAUTACION        TIPO SUB_TIPO  CANTIDAD  \
28  AJ-4626-UC5-2024            ARMAS  MUNICIONES        -       9.0   
29  AJ-4635-UC5-2024            ARMAS     PISTOLA        -       1.0   
30  AJ-4635-UC5-2024            ARMAS     PISTOLA        -       1.0   
31  AJ-4635-UC5-2024            ARMAS    REVÓLVER        -       1.0   
32  AJ-4635-UC5-2024            ARMAS  MUNICIONES        -       6.0   
33  AJ-4635-UC5-2024            ARMAS  MUNICIONES        -      13.0   
34  PS-4693-USU-2024            ARMA

### INCAUTACIONES DIVISA

In [ ]:
# Paso 1: Cargar datos iniciales
excel_bajada_divisa = pd.read_excel("bajadas/bajada_divisas_secuestradas.xls")
print(f"Total de registros cargados: {len(excel_bajada_divisa)}")

excel_bajada_procedimientos_generales['PARTE_ANIO'] = excel_bajada_procedimientos_generales['NUMERO_PARTE'].astype(str) + "/" + excel_bajada_procedimientos_generales['ANIO_PARTE'].astype(str)
excel_bajada_divisa['PARTE_ANIO'] = excel_bajada_divisa['NUMERO_PARTE'].astype(str) + "/" + excel_bajada_divisa['ANIO_PARTE'].astype(str)

# Hacer el merge para traer la columna TIPO_CAUSA_INTERNA a excel_bajada_divisa
excel_bajada_divisa = pd.merge(excel_bajada_divisa, 
                               excel_bajada_procedimientos_generales[['PARTE_ANIO', 'TIPO_CAUSA_INTERNA']], 
                               on='PARTE_ANIO', 
                               how='left')

excel_bajada_divisa['UID'] = excel_bajada_divisa.apply(generar_uid_sigpol, axis=1)


# Agrupar por 'UID' (y 'TIPO' si se necesita) para sumar las cantidades
cantidad_por_uid = excel_bajada_divisa.groupby(['UID', 'TIPO_DIVISA'], as_index=False)['TOTAL_DIVISAS_SECUESTRADAS'].sum()

# Paso 2: Crear df_divisa con las cantidades sumadas
df_divisa = pd.DataFrame()
df_divisa['UID'] = cantidad_por_uid['UID']
df_divisa['TIPO_INCAUTACION'] = "DIVISAS"
df_divisa['TIPO'] = cantidad_por_uid['TIPO_DIVISA']
df_divisa['SUB_TIPO'] = "-"
df_divisa['CANTIDAD'] = cantidad_por_uid['TOTAL_DIVISAS_SECUESTRADAS']
df_divisa['MEDIDAS'] = "-"
df_divisa['AFORO'] = "-"
df_divisa['OBSERVACIONES'] = "-"

# Mostrar el resultado final
print(df_divisa.head())


df_divisa_final = df_divisa[df_divisa['UID'].isin(df_geog_final['UID'])]
print(f"Cantidad de registros con UID coincidente: {len(uid_coincidentes)}")

Total de registros cargados: 85
                UID TIPO_INCAUTACION                  TIPO SUB_TIPO  CANTIDAD  \
0  AJ-4201-EZE-2024          DIVISAS  DÓLAR ESTADOUNIDENSE        -        10   
1  AJ-4201-EZE-2024          DIVISAS                 EURO         -      2087   
2  AJ-4201-EZE-2024          DIVISAS        PESO ARGENTINO        -     56904   
3  AJ-4201-EZE-2024          DIVISAS                  REAL        -        39   
4  AJ-4267-UC1-2024          DIVISAS  DÓLAR ESTADOUNIDENSE        -      1474   

  MEDIDAS AFORO OBSERVACIONES  
0       -     -             -  
1       -     -             -  
2       -     -             -  
3       -     -             -  
4       -     -             -  
Cantidad de registros con UID coincidente: 10


### INCAUTACIONES OBJETOS

In [ ]:
# Paso 1: Cargar datos iniciales
excel_bajada_objetos = pd.read_excel("bajadas/bajada_objetos.xls")
print(f"Total de registros cargados: {len(excel_bajada_objetos)}")

excel_bajada_objetos['TIPO_CAUSA_INTERNA'] = excel_bajada_objetos.apply( procesar_tipo_causa_interna, axis=1)
excel_bajada_objetos['UID'] = excel_bajada_objetos.apply(generar_uid_sigpol, axis=1)



# Paso 2: Crear df_divisa con las cantidades sumadas
df_objetos = pd.DataFrame()
df_objetos['UID'] = excel_bajada_objetos['UID']
df_objetos['TIPO_INCAUTACION'] = "MERCADERIA"
df_objetos['TIPO'] = excel_bajada_objetos.apply(clasificar_tipo_objeto, axis=1)
df_objetos['SUB_TIPO'] = "-"
df_objetos['CANTIDAD'] = excel_bajada_objetos['CANTIDAD']
df_objetos['MEDIDAS'] = "-"
df_objetos['AFORO'] = "-"
df_objetos['OBSERVACIONES'] = "-"

# Mostrar el resultado final
print(df_objetos.head())


df_objetos_final = df_objetos[df_objetos['UID'].isin(df_geog_final['UID'])]
print(f"Cantidad de registros con UID coincidente: {len(uid_coincidentes)}")

Total de registros cargados: 285
                UID TIPO_INCAUTACION                        TIPO SUB_TIPO  \
0  PS-4150-EZE-2024       MERCADERIA  COMUNICACIONES - CELULARES        -   
1  PS-4158-COR-2024       MERCADERIA                 OTRO - ROBO        -   
2  PS-4160-NEU-2024       MERCADERIA         OTRO - NARCOTRAFICO        -   
3  PS-4161-COR-2024       MERCADERIA       PERSONALES - PERFUMES        -   
4  PS-4161-COR-2024       MERCADERIA   PERSONALES - INDUMENTARIA        -   

   CANTIDAD MEDIDAS AFORO OBSERVACIONES  
0       1.0       -     -             -  
1       1.0       -     -             -  
2       1.0       -     -             -  
3       1.0       -     -             -  
4       1.0       -     -             -  
Cantidad de registros con UID coincidente: 0


### INCAUTACIONES NARCOTRAFICO

# OPERACIONES

### ORDEN DE SERVICIOS

In [ ]:
excel_bajada_operaciones = load_workbook("bajadas/operaciones.xlsx")
hoja_geog = excel_bajada_operaciones['GEOG. PROCEDIMIENTO']

df_operaciones = leer_excel_a_df(hoja_geog)
df_operaciones = df_operaciones.dropna(subset=['ID_PROCEDIMIENTO'], how='any')
df_operaciones["ID_PROCEDIMIENTO"] = df_operaciones["ID_PROCEDIMIENTO"].apply(colocar_guion_espacio)
df_operaciones["ID_OPERATIVO"] = df_operaciones["ID_PROCEDIMIENTO"].apply(formatear_contador)
df_operaciones["UID"] = df_operaciones.apply(generar_uid_operaciones , axis=1)
df_operaciones['FECHA'] = pd.to_datetime(df_operaciones['FECHA'], errors='coerce')

df_operaciones = df_operaciones[[ "UID",'FUERZA_INTERVINIENTE', 'ID_OPERATIVO', 'ID_PROCEDIMIENTO',
                                     'UNIDAD_INTERVINIENTE', 'DESCRIPCIÓN', 'TIPO_INTERVENCION',
                                     'PROVINCIA', 'DEPARTAMENTO O PARTIDO', 'LOCALIDAD', 'DIRECCION',
                                     'ZONA_SEGURIDAD_FRONTERAS', 'PASO_FRONTERIZO', 'LATITUD', 'LONGITUD',
                                     'FECHA', 'HORA', 'OTRAS AGENCIAS INTERVINIENTES', 'Observaciones - Detalles']]


print(f"Cantidad de registros en df_otros_delitos: {len(df_operaciones)}")

# # Aplicar la función para identificar las filas que son diferentes

# df_operaciones = colocar_contador(df_operaciones , df_base_operaciones)




### GENERACIÓN DE INFORME

In [ ]:

# Cargar la plantilla de Excel
wb = load_workbook('Planilla_modelo.xlsx')
hoja_geog= wb['GEOG. PROCEDIMIENTO']
hoja_vehiculo = wb['VEHI. Y PERSO. CONTROLADAS']
hoja_personal= wb['PERSONAL Y ELEMENTOS AFECTADOS']
hoja_detenidos= wb['DETENIDOS Y APRENDIDOS']
hoja_incautaciones = wb['INCAUTACIONES']
hoja_trata= wb['TRATA O TRAFIC PERSONAS']
hoja_otros_delitos= wb['OTROS DELITOS']
hoja_otros_eventos= wb['OTROS EVENTOS']
hoja_fallecidos= wb['FALLECIDOS']
hoja_abatidos= wb['ABATIDOS']


# Cargar la plantilla de Excel
wb = load_workbook('Planilla_modelo.xlsx')

# Escribir el DataFrame en la hoja de trabajo
df_geog_final.to_excel(hoja_geog, header=False, index=False, startrow=3)

# Guardar la planilla modelo con los datos llenados
wb.save(f'Informe_semanal_{datetime.now().strftime("%Y%m%d")}.xlsx')


